# Bucket Brigade

**Authors:**
- Tomer Yanay
- Yogev Matalon
- Noam Tor

In [20]:
import pulp
import time
import numpy as np
import pandas as pd
import random
import simpy
from pylab import plot, show, bar
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import math
from scipy import stats

# Part 1 - Create the Seed for the Genetic Algorithm

### 1.A. "Super Parent" - Linear Programming Solution

In [2]:
def lp_Heuristic(filename, sheetname, capacity_num):

#load the data
    data = pd.read_excel(filename, sheetname)
    orders = data[[0,1,2,3,4,5]]
    sum_products = orders.apply(sum, axis=1)
    if capacity_num == 1:
        capacity = data['capacity1'][0]
    else:
        capacity = data['capacity2'][0]
    i_orders = list(orders.index.values)
    i_batch = list(orders.index.values)
    i_order_batch = []
    for order in i_orders:
        for batch in i_batch:
            i_order_batch.append((batch, order))
            
#initialise the model
    model = pulp.LpProblem('The orders to batches problem', pulp.LpMinimize)


# initialise the variables
    x = pulp.LpVariable.dict('x', i_batch, lowBound =0, upBound = 1, cat = pulp.LpInteger) #x_1 = 1 if we use bin 1, 0 else
    y = pulp.LpVariable.dicts('y', i_order_batch ,lowBound = 0, upBound = 1, cat = pulp.LpInteger) #y_(1,1)- 1 if order 1 in batch 1, 0 else 

# create the objective
    model += pulp.lpSum( [x[batch] for batch in i_batch])

# First constraint: For every item, the sum of bins in which it appears must be 1
    for order in i_orders:
        model += pulp.lpSum([y[(batch, order)] for batch in i_batch]) == 1

# Second constraint: desicion variables connection
    for order in i_orders:
        for batch in i_batch:
            model += x[batch] >= y[(batch, order)] 

# third constraint: capacity
    for batch in i_batch:
        model += sum([sum_products[order]*y[(batch, order)] for order in i_orders]) <= capacity
    
# Solve the optimization.
    start_time = time.time()
    model.solve()

# return results as chromozom
    results = {}
    for key in y.keys():
        if y[key].value()==1:
            results[key[1]]=key[0]
    lst = [None]*20
    for key in results.keys():
        lst[key] = results[key]
    return lst

lp_Heuristic('input.xlsx', '20 orders new format', 1)

[11, 6, 14, 17, 14, 7, 3, 3, 3, 6, 14, 3, 17, 14, 17, 14, 7, 11, 11, 6]

### 1.B. Random-Greedy Seed Generator

In [3]:
def create_naive_solution(filename = 'input.xlsx', sheetname='20 orders new format' , capacity = 1):

# load dada and shuffle the orders
    data = pd.read_excel(filename, sheetname)
    orders = data.drop(['capacity1', 'capacity2'], axis=1)
    capacity = data['capacity'+ str(capacity)][0]
    orders = orders.sample(frac=1).reset_index(drop=False) #randomly shuffle the orders to get new solution each time
    old_index = orders['index'] #seave the original order index
    orders = orders.drop('index', axis = 1)
    sum_products = orders.apply(sum, axis=1) #create vector with the total amount of products in each order

#unite the orders to batches 
    lst = [None]*(orders.shape[0]) #list for the final solution in chromozom format
    i=0 #variablre for the loop
    batches = [] #list of lists- will contain the batches
    while i < (len(sum_products)):
        if sum_products[i] < capacity:
            batch = [i]
            product_counter = sum_products[i]
            for j in range(i+1, len(sum_products), 1):
                if product_counter + sum_products[j] <= capacity:
                    product_counter += sum_products[j]
                    batch.append(j)
                    sum_products[j]= capacity+1
            batches.append(batch)
        i += 1

# wtire the solution in chromozom format
    for i in range(len(batches)):
        for order in batches[i]:
            lst[old_index[order]] = i
    return lst
create_naive_solution('input.xlsx', '20 orders new format', 1)

[5, 0, 1, 5, 0, 3, 0, 1, 4, 2, 0, 1, 4, 1, 2, 5, 3, 2, 1, 6]

# Part 2 - Bucket Brigade Simulation

In [4]:
class factory(object):
    """
    """
    def __init__(self, env, items_number, workers_number, collect_rate, forward_rate, back_rate, batches):
        self.env = env
        self.item = []                      #list of locations. each location is a resource class
        self.workers = []                   #list of workers (class)   
        self.batch_order = batches          #list of batches. each batch is a a dictionary- item:quantity
        self.finish_all = env.event()       #event which represent that all the batches are done
        self.time_to_finish = 900             #the time the batches were ended
        self.items_number = items_number    
        for i in range(items_number):
            self.item.append(simpy.Resource(env, 1))
        for j in range(workers_number):
            self.workers.append(worker(env,j+1, collect_rate[j],forward_rate[j],back_rate[j]))

In [5]:
class box():
    def __init__(self, env,batch,worker,factory):
        self.batch = batch                  #dictinary of items and their quantity
        self.finished = env.event()         #batch is finished?
        self.status = worker                #which worker is with the batch box

        
    def create_forward(self,env,factory):
        self.status.direction = 1                #direction of the worker is forward
        i = self.status.name                    
        t_forword = max(0,np.random.normal(self.status.forward_rate[0],self.status.forward_rate[1])) #forward time
        t_collect = max(0,np.random.normal(self.status.collect_rate[0],self.status.collect_rate[1])) #collect time
        #option 1: the last worker always go forward without asking  
        if i==len(factory.workers):
            #print(self.status.location)
            if self.status.location>len(factory.item)-1:
                if len(factory.batch_order)==0:
                    factory.time_to_finish=env.now
                else:
                    env.process(self.status.create_back(env,factory))
            else:
                #print(self.status.location)
                with factory.item[self.status.location].request() as request:
                    yield request
                    self.status.location+=1
                    #1.1 with collect item
                    if self.status.location in self.batch.keys():
                        #print ("worker {} going and collect item {}".format(i,self.status.location))
                        self.status.pick = 1
                        yield env.timeout(t_forword+t_collect*(self.batch.get(self.status.location)))
                        #print ("worker {} has finished to collect item {}".format(i,self.status.location))
                        self.status.pick = 0
                        #1.1.1 batch is not over
                        if self.status.location<(list(self.batch.keys())[-1]):
                            #print("worker {} need to collect more".format(i))
                            env.process(self.create_forward(env,factory))
                        #1.1.2 batch is over. need to go back?
                        else:
                            #1.1.2.1 There is no batches & all the workers has finished their jobs
                            if len(factory.batch_order)==0 and factory.workers[i-2].finish==True:
                                #print("all the batches have been finished")
                                factory.time_to_finish = env.now
                                #factory.finish_all.succeed()
                            #1.1.2.2 there are more batches in the way. need to go back
                            else:
                                #1.1.2.2.1 the last worker want to go back. need to check the status of the i-1 worker
                                if self.status.location==factory.workers[i-2].location and factory.workers[i-2].direction==1:
                                    #print("worker {} is waiting for worker {}".format(i,i-1))   
                                    self.status.direction=0
                                elif self.status.location==factory.workers[i-2].location and factory.workers[i-2].direction==3:
                                    #print("special change")
                                    #if len(factory.batch_order)==0 and factory.workers[i-3].finish==True:
                                     #   factory.workers[i-2].finish=True
                                    env.process(change(env, factory.workers[i-1], self.status, factory,1))
                                #1.1.2.2.2 can go back
                                else:
                                    if self.status.location==(factory.workers[i-2].location+1) and factory.workers[i-2].pick==0 and factory.workers[i-2].direction==1:
                                        #print("worker {} is waiting for worker {}, who is going forward".format(i,i-1))
                                        self.status.direction=0
                                    else:
                                        #print("worker {} is going back-batch has been finished".format(i))
                                        env.process(self.status.create_back(env,factory))
                    #1.2 no need collect item. need to go forward
                    else:
                        if self.status.location<(list(self.batch.keys())[-1]):
                            yield env.timeout(t_forword)
                            #print("worker {} is in item {} without collecting".format(i,self.status.location))
                            env.process(self.create_forward(env,factory))

        #option 2: other workers- can be block while going forward
        else:
            with factory.item[self.status.location].request() as request:
                yield request
                self.status.location+=1
                #2.1 with collect item
                if self.status.location in self.batch.keys():
                    #print ("worker {} is going to collect item {}".format(i,self.status.location))
                    yield env.timeout(t_forword)
                    #2.1.1 worker is block because the other worker coming back
                    if factory.workers[i].location==self.status.location and (factory.workers[i].direction==0 or factory.workers[i].direction==3):
                        #print("worker {} and worker {} are in same location to change buckets".format(i,i+1))
                        env.process(change(env, factory.workers[i], self.status, factory,1))
                    elif factory.workers[i].location==self.status.location and factory.workers[i].direction==1 and factory.workers[i].pick==1 and factory.workers[i].location==(list(factory.workers[i].with_box.batch.keys())[-1]):
                        #print("worker {} with a special wating becaus worker {} in his last item".format(i,i+1))
                        self.status.direction=3
                        if len(factory.batch_order)==0 and factory.workers[i-2].finish==True:
                            factory.workers[i-1].finish=True
                    #2.1.2 worker can collect
                    else:
                        self.status.pick = 1
                        k = self.batch.get(self.status.location)
                        if k==None:
                            time_C=0
                        else:
                            time_C=k
                        yield env.timeout(t_collect*time_C)
                        #print ("worker {} has finished to collect item {}".format(i,self.status.location))
                        self.status.pick = 0
                        #2.1.2.1 need to change after collecting?
                        if factory.workers[i].location==self.status.location and factory.workers[i].direction==0:
                            #print("worker {} and worker {} are in same location to change buckets".format(i,i+1))
                            env.process(change(env, factory.workers[i], self.status, factory,2))
                        #2.1.2.2 no change
                        else:
                            #2.1.2.2.1 batch is not over
                            if self.status.location<(list(self.batch.keys())[-1]):
                                #print("worker {} need to collect more".format(i))
                                #2.1.2.2.1.1 worker i need to wait bacause worker i+1 is going back
                                if factory.workers[i].direction==0 or factory.workers[i].direction==3:
                                    if factory.workers[i].direction==0:
                                        #print("worker {} is waiting for worker {} who is going backward".format(i,i+1))
                                        self.status.direction=3
                                    else:
                                        #print("worker {} start going forward".format(i))
                                        env.process(self.create_forward(env,factory))
                                #2.1.2.2.1.2 worker i is going forward
                                else: 
                                    #print("worker {} start going forward".format(i))
                                    env.process(self.create_forward(env,factory))
                            #2.1.2.2.2 batch is over. need to go back?
                            else:
                                #2.1.2.2.2.1 There is no batches & the workers has finished their jobs
                                if len(factory.batch_order)==0:
                                    if i==1:
                                        self.status.finish=True
                                        self.status.location=-i
                                        #print("There are no work for worker {}".format(i))
                                    else: 
                                        if factory.workers[i-2].finish==True:
                                            #print("There are no work for worker {}".format(i))
                                            self.status.finish=True
                                            self.status.location=-i
                                        #the i-1 workers is still working. need to go back
                                        else:
                                            #print("worker {} is going back-batch has been finished".format(i))
                                            env.process(self.status.create_back(env,factory))
                                #2.1.2.2.2.2 there are more batches in the way. need to go back
                                else:
                                    if i==1:
                                        #print("worker {} is going back-batch has been finished".format(i))
                                        env.process(self.status.create_back(env,factory))
                                    else:
                                        #2.1.2.2.2.2.1 the worker want to go back. need to check the status of the i-1 worker
                                        if self.status.location==factory.workers[i-2].location and factory.workers[i-2].direction==1:
                                            #print("worker {} and worker {} are in same location to change buckets".format(i,i+1))
                                            env.process(change(env, factory.workers[i], self.status, factory,1))
                                        #2.1.2.2.2.2.2 can go back?
                                        else:
                                            #2.1.2.2.2.2.2.1 need to wait
                                            if self.status.location==(factory.workers[i-2].location+1) and factory.workers[i-2].pick==0 and factory.workers[i-2].direction==1:
                                                #print("worker {} is waiting for worker {}, who is going forward".format(i,i-1))
                                                self.status.direction=3
                                            #2.1.2.2.2.2.2.2 can go back
                                            else:
                                                #print("worker {} is going back-batch has been finished".format(i))
                                                env.process(self.status.create_back(env,factory))
                #2.2 no need to collect item. need to go forward
                else:
                    yield env.timeout(t_forword)
                    #print("worker {} is in item {} without collecting".format(i,self.status.location))
                    #2.2.1 worker is block because the other worker coming back
                    if factory.workers[i].location==self.status.location and (factory.workers[i].direction==0 or factory.workers[i].direction==3):
                        #print("worker {} and worker {} are in same location to change buckets".format(i,i+1))
                        env.process(change(env, factory.workers[i], self.status, factory,2))
                    #2.2.2 worker chaeck if he can go forward to the next item
                    else:
                        #print("worker {} need to collect more".format(i))
                        #2.2.2.1 worker i need to wait and can't go forward
                        if factory.workers[i].direction==0:
                            #print("worker {} is waiting for worker {} who is going backward".format(i,i+1))
                            self.status.direction=0
                        #2.2.2.2 worker i is going forward
                        else:
                            #print("worker {} start going forward".format(i))
                            env.process(self.create_forward(env,factory))

In [6]:
class worker():
    def __init__(self, env, name, collect_rate,forward_rate,back_rate):
        self.name = name
        self.collect_rate = collect_rate
        self.forward_rate = forward_rate
        self.back_rate = back_rate             
        self.location = 0                      #worker location by axis x
        self.finish = False                    #if the worker has finished his job
        self.with_box = None
        self.direction = 1                    #1 forward/ 0 backward
        self.pick = 0                         #is picking?
        self.back_time = 0
        
    def create_back(self, env, factory):
        self.direction = 0                            #direction of the worker is backward
        i = self.name
        t_backward1 = max(0,np.random.normal(self.back_rate[0],self.back_rate[1]))
        if factory.workers[i-2].direction==0:
            t_backward = max(t_backward1,factory.workers[i-2].back_time)
        else:
            t_backward=t_backward1
        self.back_time = t_backward
        #option 1: the first worker always go back, can't be blocked 
        if i==1:
            yield env.timeout(t_backward)
            self.location-=1
            #1.1 checking every step back if we have batches
            if len(factory.batch_order)==0:
                self.finish=True
                self.location=-i
                #print("There are no work for worker {}".format(i))
            #1.2 we have batches...
            else:
                #1.2.1 he's in location 0- need to pick item
                if self.location==0:
                    if len(factory.batch_order)>0:
                        new_batch = factory.batch_order.pop(0)
                        bucket = box(env, new_batch, self, factory)
                        #print("worker {} took new bucket: {}".format(self.name,new_batch))
                        self.with_box = bucket
                        env.process(bucket.create_forward(env,factory))
                    else:
                        factory.time_to_finish = env.now
                #1.2.2 not in location 0- need to go back
                else:
                    #print("worker {} need is going back".format(i))
                    env.process(self.create_back(env, factory))
        #option 2: other workers are going back, can be blocked or to change buckets           
        else:
            yield env.timeout(t_backward)
            self.location-=1
            #2.1 checking every step back if we have batches and worker i-1 is finished
            if len(factory.batch_order)==0 and factory.workers[i-2].finish==True:
                self.finish=True
                self.location=-i
                #print("There are no work for worker {}".format(i))
            #2.2 we have batches...    
            else:
                #2.2.1 he's in location 0- need to pick item
                if self.location==0:
                    if len(factory.batch_order)>0:
                        new_batch = factory.batch_order.pop(0)
                        bucket = box(env, new_batch, self, factory)
                        #print("worker {} took new bucket: {}".format(self.name,new_batch))
                        self.with_box = bucket
                        env.process(bucket.create_forward(env,factory))
                    else:
                        #print("all the batches have been finished")
                        factory.time_to_finish = env.now
                        #factory.finish_all.succeed()
                #2.2.2 not in location 0- need to go back/block/change buckets
                else:
                    #2.2.2.1 checking the status of the i-1 worker
                    if factory.workers[i-2].location==self.location and (factory.workers[i-2].direction==1 or factory.workers[i-2].direction==3):
                        #print("worker {} and worker {} are in same location to change buckets".format(i,i-1))
                        #2.2.2.1.1 changing now or block
                        if factory.workers[i-2].pick==0:
                            #print("worker {} and worker {} change buckets".format(i,i-1))
                            env.process(change(env, self, factory.workers[i-2], factory,2))                
                        #2.2.2.1.2 block and i-1 worker will trigger the changing
                        #else:
                            #print("worker {} is still picking. worker {} back block. the change is delay".format(i-1,i))
                    #2.2.2.2 we can go backward
                    else:
                        if i==2:
                            if self.location==factory.workers[i-2].location:
                                #print("need to change")
                                env.process(change(env, self, factory.workers[i-2], factory,2))
                            else:
                                #print("worker {} need is going back".format(i))
                                env.process(self.create_back(env, factory))
                        else:
                            if factory.workers[i-2].location==factory.workers[i-3] and self.location==(factory.workers[i-2]+1):
                                #print("worker {} need to wait becuase the other workers are changing buckets".format(i))
                                self.direction=4
                            else:
                                env.process(self.create_back(env, factory))

In [7]:
def change(env, worker1, worker2, factory, state):
    #state=1 :the change happens before collecting in the current location
    if state==1:
        worker1.with_box = worker2.with_box
        worker1.with_box.status = worker1
        env.process(worker2.create_back(env,factory))
        worker1.pick = 1
        t_collect = max(0,np.random.normal(worker1.collect_rate[0],worker1.collect_rate[1])) #collect time
        worker1.direction=1
        yield env.timeout(t_collect*(worker1.with_box.batch.get(worker1.location)))
        #Same conditions as 1.1. in going forward
        #print ("worker {} has finished to collect item {}".format(worker1.name,worker1.location))
        worker1.pick = 0
        if worker1.location<(list(worker1.with_box.batch.keys())[-1]):
            #print("worker {} need to collect more".format(worker1.name))
            if worker1.location<(list(worker1.with_box.batch.keys())[-1]):
                env.process(worker1.with_box.create_forward(env,factory))
        else:
            #print(factory.workers[worker1.name-2].finish,factory.workers[worker1.name-2].name)
            if len(factory.batch_order)==0 and factory.workers[worker1.name-2].finish==True:
                #print("all the batches have been finished")
                factory.time_to_finish = env.now
                #factory.finish_all.succeed()
                #1.1.2.2 there are more batches in the way. need to go back
            else:
                #1.1.2.2.1 the last worker want to go back. need to check the status of the i-1 worker
                if worker1.location==factory.workers[worker1.name-2].location and factory.workers[worker1.name-2].direction==1:
                    #print("worker {} is waiting for worker {}".format(worker1.name,worker1.name-1))   
                    worker1.direction=3
                #1.1.2.2.2 can go back
                else:
                    if worker1.location==(factory.workers[worker1.name-2].location+1) and factory.workers[worker1.name-2].pick==0 and factory.workers[worker1.name-2].direction==1:
                        #print("worker {} is waiting for worker {}, who is going forward".format(worker1.name,worker1.name-1))
                        worker1.direction=3
                    else:
                        #print("worker {} is going back-batch has been finished".format(worker1.name))
                        env.process(worker1.create_back(env,factory))
    #state=2 :the change happens after collecting in the current location
    else:
        worker1.with_box = worker2.with_box
        worker1.with_box.status = worker1
        env.process(worker2.create_back(env,factory))
        if worker1.location<(list(worker1.with_box.batch.keys())[-1]):
            #print("worker {} need to collect more".format(worker1.name))
            if worker1.name!=len(factory.workers):
                if worker1.location+1==factory.workers[worker1.name].location and factory.workers[worker1.name].direction==1 and factory.workers[worker1.name].pick==1 and factory.workers[worker1.name].location==(list(factory.workers[worker1.name].with_box.batch.keys())[-1]):
                    worker1.direction=3
                else:
                    if factory.workers[worker1.name].direction==4:
                        worker1.location+=1
                        yield env.timeout(1)
                        if worker1.location in worker1.with_box.batch.keys():
                            new_state=1
                        else:
                            new_state=2
                        env.process(change(env, factory.workers[worker1.name], worker1, factory,new_state))
                    else:
                        if worker1.location<(list(worker1.with_box.batch.keys())[-1]):
                            env.process(worker1.with_box.create_forward(env,factory))
            else:
                if worker1.location<(list(worker1.with_box.batch.keys())[-1]):
                    env.process(worker1.with_box.create_forward(env,factory))
        else:
            #print(factory.workers[worker1.name-2].finish,factory.workers[worker1.name-2].name)
            if len(factory.batch_order)==0 and factory.workers[worker1.name-2].finish==True:
                #print("all the batches have been finished")
                factory.time_to_finish = env.now
                #factory.finish_all.succeed()
                #1.1.2.2 there are more batches in the way. need to go back
            else:
                #1.1.2.2.1 the last worker want to go back. need to check the status of the i-1 worker
                if worker1.location==factory.workers[worker1.name-2].location and factory.workers[worker1.name-2].direction==1:
                    #print("worker {} is waiting for worker {}".format(worker1.name,worker1.name-1))   
                    worker1.direction=3
                #1.1.2.2.2 can go back
                else:
                    if worker1.location==(factory.workers[worker1.name-2].location+1) and factory.workers[worker1.name-2].pick==0 and factory.workers[worker1.name-2].direction==1:
                        #print("worker {} is waiting for worker {}, who is going forward".format(worker1.name,worker1.name-1))
                        worker1.direction=3
                    else:
                        #print("worker {} is going back-batch has been finished".format(worker1.name))
                        env.process(worker1.create_back(env,factory))   

In [9]:
def setup(env,factory,workers_number, collect_rate, forward_rate, back_rate):
    """
    """
    for w in range(workers_number,0,-1):
        new_worker = factory.workers[w-1]
        new_batch = factory.batch_order.pop(0)
        bucket = box(env, new_batch, new_worker, factory)
        #print("worker {} start working at time {}".format(w, env.now))
        new_worker.with_box = bucket
        env.process(bucket.create_forward(env,factory))
        yield env.timeout(0.01)

In [10]:
def main_sim(items_number, workers_number, collect_rate, forward_rate, back_rate, batches):    
    # Create an environment and start the setup process
    RANDOM_SEED =  42
    # ----------Setup and start the simulation-------------------
    resultList = []
    for i in range(50):
        env = simpy.Environment()
        # Create the factory
        Factory = factory(env, items_number, workers_number, collect_rate, forward_rate, back_rate, batches[:])
        # start the simulation
        #np.random.seed(RANDOM_SEED)  # This helps reproducing the results

        env.process(setup(env,Factory,workers_number, collect_rate, forward_rate, back_rate))
        # Execute!
        #env.run(until=Factory.finish_all)

        env.run()
        
        resultList.append(Factory.time_to_finish)
    
    avarageList = []
    for j in range(len(resultList)):
        if resultList[j] != 900:
            avarageList.append(resultList[j])
    
    if len(avarageList) != 0:
        return np.average(avarageList)
    else:
        return np.inf

In [11]:
main_sim(6,2,[(1.1, 0.2), (1, 0.2)],[(1, 0.2), (1, 0.2)],[(0.5, 0.1), (0.5, 0.1)],[{1: 7, 2: 13, 3: 11, 4: 13, 5: 3, 6: 9},
 {1: 14, 2: 16, 3: 2, 4: 12, 5: 7, 6: 7},
 {1: 2, 2: 9, 3: 12, 4: 11, 5: 7, 6: 13},
 {1: 7, 2: 12, 3: 5, 4: 8, 5: 7, 6: 19},
 {1: 8, 2: 7, 3: 15, 4: 1, 5: 11, 6: 14},
 {1: 12, 2: 15, 3: 7, 4: 3, 5: 7, 6: 14},
 {1: 7, 2: 4, 3: 1, 4: 2, 5: 2, 6: 0}])

232.31808383043153

## Simulation Parameter - Read Excel File

In [12]:
def getParameters_fromExcel(filename, sheetname, capacity = 1):
    '''
    :param: filename, sheetname: the name of the excel file and the relvant sheet for the given simulation.
    :param: capacity: can be 1 or 2 (each sheet has 2 capacity options)
    :return: the orders DataFrame, and the capacity integer 
    '''
    data = pd.read_excel(filename, sheetname)
    orders = data.drop(['capacity1', 'capacity2'], axis=1)
    capacity = data['capacity'+ str(capacity)][0]
    sum_products = orders.apply(sum, axis=1) #create vector with the total amount of products in each order
    return orders, capacity

------------


# Part 3 - The Genetic Algorithm

## Chromozom to Buckets ("Batches") and their orders

In [13]:
def chromozom_to_batches(chromozom, data):
    # deal with empty buckets
    uniqueSortedChrom = list(np.unique(chromozom)) 
    for gene in range(len(chromozom)):
        chromozom[gene] = uniqueSortedChrom.index(chromozom[gene])        
    
    # the number of batches is similar to the maximum number in the buckets (+1, because we star with zero)
    batches = [{item:0 for item in range(1, data.shape[1]+1, 1)} for d in range(max(chromozom)+1)]  # batches is a list of dictionaries. [{itemType : quantity}, {} ...]. The last dictionary represents the first bucket to be done.
    #print (batches)
    #print (data)
    
    for j in range(len(chromozom)):
        for i in range(data.shape[1]):
            batches[chromozom[j]][i+1] += data[i][j] 
    
    # delete items with zero value in all dictionaries    
    for bucket in range(len(batches)):
        for item in range(1, data.shape[1], 1):
            if batches[bucket][item] == 0:
                del batches[bucket][item]
    return batches

# TEST ONLY !
data, capacity = getParameters_fromExcel('input.xlsx', '20 orders new format', 1)
print (data)
chromozom_to_batches([0, 3, 4, 2, 0, 5, 4, 0, 5, 4, 0, 3, 1, 1, 2, 2, 1, 6, 3, 3], data)

    0  1  2  3  4  5
0   4  0  6  7  0  6
1   2  2  3  3  1  6
2   0  5  3  0  1  4
3   1  6  2  0  4  6
4   2  7  1  1  3  0
5   4  8  6  1  7  7
6   0  0  8  1  3  5
7   0  5  4  0  0  2
8   8  7  1  2  0  7
9   8  2  4  0  7  5
10  1  1  0  5  0  1
11  0  0  0  1  5  1
12  5  5  0  7  0  6
13  1  5  0  2  0  0
14  1  2  7  6  2  0
15  0  1  3  5  1  7
16  8  6  2  3  7  1
17  7  4  1  2  2  0
18  5  6  0  4  0  5
19  0  4  2  0  1  7


[{1: 7, 2: 13, 3: 11, 4: 13, 5: 3, 6: 9},
 {1: 14, 2: 16, 3: 2, 4: 12, 5: 7, 6: 7},
 {1: 2, 2: 9, 3: 12, 4: 11, 5: 7, 6: 13},
 {1: 7, 2: 12, 3: 5, 4: 8, 5: 7, 6: 19},
 {1: 8, 2: 7, 3: 15, 4: 1, 5: 11, 6: 14},
 {1: 12, 2: 15, 3: 7, 4: 3, 5: 7, 6: 14},
 {1: 7, 2: 4, 3: 1, 4: 2, 5: 2, 6: 0}]

# Genetic algorithem

In [14]:
simCounter = 0

def f(chromozom, parameters):
    global simCounter
    # returns the avarage time from the simulations, due to inputs x
    batches = chromozom_to_batches(chromozom, parameters['orders'])
    if batches == []:
        print ("batches is empty!!")
        print ("the chromozom is: ", chromozom)
        print ("orders parameters are: ", parameters['orders'])
        print ("retrying creating batches")
        batches = chromozom_to_batches(chromozom, parameters['orders'])
#     print ("Batches for simulation :", batches)
    
#     print ("\nParameters for simulation: ")
#     print ("items_number =", parameters['orders'].shape[1],
#             "\nworkers_number =",  parameters['pickersNum'],
#             "collect_rate =", parameters['pickersPerf']['tp'], "\nforward_rate =", parameters['pickersPerf']['tf'],
#             "\nback_rate =", parameters['pickersPerf']['tb'],
#             "\nbatches =", batches)
    batches_for_sim = batches[:]
    try:
        if resultMemory.get(str(chromozom)):
            return resultMemory.get(str(chromozom))
            
        else:
            sim = main_sim(items_number = parameters['orders'].shape[1],
                            workers_number = parameters['pickersNum'],
                            collect_rate = parameters['pickersPerf']['tp'],
                            forward_rate = parameters['pickersPerf']['tf'],
                            back_rate = parameters['pickersPerf']['tb'],
                            batches = batches_for_sim)
    except:
        print ("\n\nERROR!!!\nTHE KILLING CHROMOZOM:",chromozom)
        print ("THE BATCHES DICTIONARY:",batches)
        print ("THE BATCHES FOR SIMULATION DICTIONARY:",batches_for_sim)
        print ("THE PARAMETERS:")
        print (       'Number of Pickers :', parameters['pickersNum'],
                      '\nPickers Performance :', parameters['pickersPerf'],
                       '\nCapacity :', parameters['capacity'],
                       '\nNumber of Item Types :', orders.shape[1],
                       '\nNumber of orders :', orders.shape[0],
                      '\n\nOrders:\n', parameters['orders'])
        raise
    if sim == 900:
        if simCounter < 3:
            simCounter += 1
            return f(chromozom, parameters)
        else:
            simCounter = 0
            return np.inf
    
    return sim 


In [15]:
resultMemory = {} # A dictionary that remembers that value for every chromozom. The chromozom is converted to a string (list are not hushable).

########################################################################################
# Convert simulation arrangement inputs to a chromozom
def x_to_chromozom(x):
    # The returned chromozom is an array, that represents the arrangement.
    # Position j in the array represent order j, and the number in this position represent the number of the box for this order.
    chromozom = []
    return chromozom
########################################################################################



# Generate a random chromozom
def random_chromozom(ordersLen):
    '''
    :param ordersLen: the length of the orders array.
    :return: a random chromozom.
    '''
    J = ordersLen # get the number of orders
    for i in range(J):
        temp[i]= random.randint(0, J)
    return x_to_chromozom(temp)


# Check if the chromozom is valid
def validate_chromozom(chromozom, parameters):
    buckets = chromozom_to_batches(chromozom, parameters['orders'])
    for bucket in buckets:
        totalItemsNum = 0
        for item in bucket.keys():
            totalItemsNum += bucket[item]
        if totalItemsNum > parameters['capacity']:
            return False
    return True


#Create the initial Population
def init_pop(num_of_population, parameters):
    list_of_chromozoms = []
#     for i in range(int(num_of_population * 0.5)):
#         list_of_chromozoms.append(random_chromozom())
    for i in range(int(num_of_population * 1.0)):
        #print (parameters['sheetName'], parameters['capacityOption'])
        list_of_chromozoms.append(create_naive_solution(filename = parameters['excelFile'],sheetname=parameters['sheetName'], capacity=parameters['capacityOption']))
    list_of_chromozoms.append(lp_Heuristic(filename = parameters['excelFile'],sheetname = parameters['sheetName'], capacity_num=parameters['capacityOption']))  # Add the LP solution to the population
    return list_of_chromozoms



# Calculate the Fit Function for all the given population
# returns the probability to choose each chromozom from the population
def calc_fit(pop, parameters):
    # pop - The Population. An array of all the chromozoms to calculate.
    #print ("calculating fit value for population : ", pop)
    pop_values = []  # the /values of the chromozoms in the population (according to the simulations)
    for i in range(len(pop)):
        # if the chromozom is not valid, give it a very low fit
        if not validate_chromozom(pop[i], parameters):
            #print (pop[i], "is not valid.")
            pop_values.append(0.001)
            resultMemory[str(pop[i])] = np.inf # A dictionary that remembers that value for every chromozom. The chromozom is converted to a string (list are not hushable).

        else:
            val = f(pop[i], parameters)
            pop_values.append(val)  # add to the population values list the value of the current chromozom   
            resultMemory[str(pop[i])] = val
    p_list = []  # A list of the probabilities to choose chromozom i from the population
    #print ("population simulation values:", pop_values)
    for i in range(len(pop_values)):
        p_list.append(((1 - (pop_values[i] / sum(pop_values)))/(len(pop_values) - 1)))  # We want to give higher probability to lower solution, thus the (1 - value / sum_of_values)
        ("Fit value (probability) for", i, "is: ", p_list[i])
    return p_list
    
    
def mutate(chromozom):
    prob = 5#len(chromozom) # every gene in the chromozom has a 1/J probability to be mutated
    for i in range(len(chromozom)):
        temp_random = random.randint(0, prob)
        if temp_random == 1:
            chromozom[i] += 1
#             if chromozom[i] == J:  # if the order is now in box J+1 (there are only J boxes). Notice - written in Python, boxes are from 0 to J-1
#                 chromozom[i] = 0
    return chromozom


def select_parents(pop, parameters):
    list_of_parents = []
    i = 0
    probFit = calc_fit(pop, parameters)
    while i < (len(pop) / 2):
        first_parent = pop[int(np.random.choice(len(pop), 1, replace=False, p = probFit))]  # the first parent is chosen accourding to the Fit Function Probability
        second_parent = pop[int(np.random.choice(len(pop), 1, replace=False))]  # the second parent is chosen randomly.
        if np.any(first_parent != second_parent):  # if the same chromozom was chosen twice, select again.
            i += 1
            list_of_parents.append([first_parent, second_parent])
    return list_of_parents


def crossover(chromozom1, chromozom2):
    crossPoint = np.random.randint(1, len(chromozom1))
    baby_gene = np.zeros(len(chromozom1))
    baby_gene[:crossPoint] = chromozom1[:crossPoint]
    baby_gene[crossPoint:] = chromozom2[crossPoint:]
    return baby_gene

#############################################################
def get_key(item): #function for the sort key
    item2 = []
    for i in range(len(item)):
        item2.append(int(item[i]))
    # if the chromozom is not valid
    if not validate_chromozom(item2, parameters):
        #print (item2, "is not valid.")
        return np.inf
    else:
        try:
            if resultMemory.get(str(item2)):
                return resultMemory.get(str(item2))
            else:
                return f(item2, parameters)
        except:
            print (item2)
            raise
    
#############################################################

def survival(pop, offspring, parameters):
    #print ("POPULATION:", pop)
    new_generation=[]
    num_survive=int(0.1*len(pop))  # NEED TO CHECK IF 10% IS GOOD
    sorted_pop = sorted(pop, key = get_key)
    sorted_offspring= sorted(offspring, key= get_key)
    new_pop_2 = sorted_offspring[:num_survive]
    new_pop_1 = sorted_pop[:(len(pop)-num_survive)]
    new_pop = new_pop_1+new_pop_2
    for i in range(len(new_pop)):
        new_pop[i] = list(new_pop[i])
        for j in range(len(new_pop[i])):
            new_pop[i][j] = int(new_pop[i][j])
    #print ("NEW POPULATION:", new_pop)
    return new_pop

# Part 4 - The Main Code

In [16]:
def MainGeneticAlgo(parameters):
    print("\nStarting Genetic Algorithm.")
    pop = init_pop(10, parameters)
    iter_num = 30 #number of generations
    #print ("Initial population :", pop)
    best_list=[]
    for k in range(iter_num):
        print ("Generation Number : ", k)
        parents_pairs = select_parents(pop, parameters)
        new_offspring=[]
        for i in range(len(parents_pairs)): #creates 2 child from each pair
            for j in range(2):
                new_offspring.append(mutate(crossover(parents_pairs[i][0], parents_pairs[i][1])))
        pop=survival(pop,new_offspring, parameters)
        best = np.argmin(map(lambda x: get_key(x), pop)) #save the best solution
        best_list.append(get_key(pop[best]))
        #print ("Generation Average :", np.average([get_key(chrom) for chrom in pop]))
        for chrom in pop:
            print ('Value of chromozom', chrom, ':', get_key(chrom))
        print ('the best solution is ', pop[best], 'with time', resultMemory[str(pop[best])],'\n')
#-----------------------------------------------------------------------------------------------------------------------
#create graph


# t = np.arange(0.0, float(iter_num), 1.0)
# x= range(1,iter_num+1,1)
# plt.plot(x, best_list, 'b')
# plt.ylabel('Hppiness units')
# plt.title('Gene project')
# plt.xlabel('Generetion')
# plt.grid(True)

# plt.show()

In [190]:
data = {
    'data1' :
        {
            2:
            {
                'tf' : [(1, 0.2), (1,0.2)],
                'tb': [(0.5, 0.1), (0.5, 0.1)],
                'tp' : [(1.4, 0.2), (1, 0.2)]
            },
            3:
            {
                'tf' : [(1, 0.2), (1,0.2), (1,0.2)],
                'tb': [(0.5, 0.1), (0.5, 0.1), (0.5,0.1)],
                'tp' : [(1.5, 0.2), (1.2, 0.2), (0.9,0.2)]
            },
            4:
            {
                'tf' : [(1, 0.2), (1,0.2), (1,0.2), (1, 0.2)],
                'tb': [(0.5, 0.1), (0.5, 0.1), (0.5,0.1), (0.5, 0.1)],
                'tp' : [(1.9, 0.2), (1.6, 0.2), (1.3,0.2), (0.9,0.2)]
            }
        },

    'data2' :
        {
            2:
            {
               'tf' : [(1, 0.2), (1,0.2)],
                'tb': [(0.5, 0.1), (0.5, 0.1)],
                'tp' : [(1.1, 0.2), (1, 0.2)]
            },
            3:
            {
            'tf' : [(1, 0.2), (1,0.2), (1, 0.2)],
            'tb': [(0.5, 0.1), (0.5, 0.1), (0.5,0.1)],
            'tp' : [(1.2, 0.2), (1.2, 0.2), (1.1,0.2)]
            },
            4:
            {
                'tf' : [(1, 0.2), (1,0.2), (1,0.2), (1, 0.2)],
                'tb': [(0.5, 0.1), (0.5, 0.1), (0.5,0.1), (0.5, 0.1)],
                'tp' : [(1.2, 0.2), (1.1, 0.2), (1,0.2), (1,0.2)]
            }
        }
    }


sheetParameters = {'20 orders new format': {
                                            'Number of pickers': [2],
                                            'Picker performance':['data1','data2'],                   
                                            'Total capacity' : [1,2]
                                            },
                   '50 orders new format': {
                                            'Number of pickers': [2, 3],
                                            'Picker performance': ['data1','data2'],                   
                                            'Total capacity' : [1,2]
                                            },
                   '100 orders new format': {
                                            'Number of pickers': [3, 4],
                                            'Picker performance': ['data1','data2'],
                                            'Total capacity' : [1,2]
                                            }
                  }



orders, capacity = getParameters_fromExcel('input.xlsx', '20 orders new format', 1)



# The algorithm
for sheet in sheetParameters.keys():
    for pickersNum in sheetParameters[sheet]['Number of pickers']:
        for pickersPerf in sheetParameters[sheet]['Picker performance']:
            for capacityOption in sheetParameters[sheet]['Total capacity']:
                orders, capacity = getParameters_fromExcel('input.xlsx', sheet, capacityOption)
                parameters = {
                    'excelFile' : 'input.xlsx',
                    'sheetName' : sheet,
                    'pickersNum' : pickersNum,
                    'pickersPerf' : data[pickersPerf][pickersNum],
                    'capacityOption' : capacityOption,
                    'capacity' : capacity,
                    'orders' : orders
                            }

                print ('\n\n------------------------------------\n\n',
                       'Number of Pickers :', parameters['pickersNum'],
                      '\nPickers Performance :', parameters['pickersPerf'],
                       '\nCapacity :', parameters['capacity'],
                       '\nNumber of Item Types :', orders.shape[1],
                       '\nNumber of orders :', orders.shape[0],
                      '\n\nOrders:\n', parameters['orders'])
                MainGeneticAlgo(parameters)



# optionDict = {
#                 items_number = 5  # Number of items
#                 workers_number = 3  # Number of workers
#                 collect_rate = [(1.4,0.2),(1,0.2),(1.2,0.2)]
#                 forward_rate = [(1,0.2),(1,0.2),(1,0.1)]
#                 back_rate = [(0.5,0.1),(0.5,0.1),(0.7,0.1)]
#                 batches = [{1:6,2:2,3:9,4:10,5:1},{1:1,2:11,3:5,5:5},{2:3,3:10,4:10}]

#             }

# MainGeneticAlgo(

# )



------------------------------------

 Number of Pickers : 2 
Pickers Performance : {'tf': [(1, 0.2), (1, 0.2)], 'tb': [(0.5, 0.1), (0.5, 0.1)], 'tp': [(1.4, 0.2), (1, 0.2)]} 
Capacity : 60.0 
Number of Item Types : 6 
Number of orders : 20 

Orders:
     0  1  2  3  4  5
0   4  0  6  7  0  6
1   2  2  3  3  1  6
2   0  5  3  0  1  4
3   1  6  2  0  4  6
4   2  7  1  1  3  0
5   4  8  6  1  7  7
6   0  0  8  1  3  5
7   0  5  4  0  0  2
8   8  7  1  2  0  7
9   8  2  4  0  7  5
10  1  1  0  5  0  1
11  0  0  0  1  5  1
12  5  5  0  7  0  6
13  1  5  0  2  0  0
14  1  2  7  6  2  0
15  0  1  3  5  1  7
16  8  6  2  3  7  1
17  7  4  1  2  2  0
18  5  6  0  4  0  5
19  0  4  2  0  1  7

Starting Genetic Algorithm.
Generation Number :  0
Value of chromozom [4, 0, 3, 3, 2, 5, 0, 0, 1, 1, 1, 0, 5, 0, 2, 2, 3, 6, 4, 4] : 260.384750315
Value of chromozom [4, 5, 4, 0, 5, 3, 0, 0, 1, 1, 1, 2, 5, 0, 3, 4, 2, 6, 2, 6] : 264.82654275
Value of chromozom [4, 3, 5, 3, 3, 5, 0, 4, 1, 0, 3, 2, 4, 1, 

Value of chromozom [3, 0, 1, 5, 3, 2, 6, 4, 1, 0, 2, 0, 5, 0, 5, 1, 4, 2, 4, 3] : 271.254686098
Value of chromozom [3, 5, 2, 5, 3, 0, 2, 2, 6, 1, 2, 1, 4, 2, 4, 5, 0, 4, 1, 3] : 271.367062509
Value of chromozom [4, 1, 0, 6, 3, 0, 3, 1, 2, 5, 0, 1, 5, 1, 6, 1, 3, 2, 4, 2] : 273.923336566
Value of chromozom [0, 2, 4, 0, 5, 6, 1, 2, 3, 1, 4, 4, 2, 2, 0, 3, 4, 3, 5, 1] : 275.845881003
Value of chromozom [4, 1, 0, 6, 3, 0, 4, 1, 2, 5, 0, 1, 5, 1, 6, 3, 4, 4, 6, 1] : inf
the best solution is  [4, 0, 3, 3, 2, 5, 0, 0, 1, 1, 1, 0, 5, 0, 2, 2, 3, 6, 4, 4] with time 260.384750315 

Generation Number :  12
Value of chromozom [4, 0, 3, 3, 2, 5, 0, 0, 1, 1, 1, 0, 5, 0, 2, 2, 3, 6, 4, 4] : 260.384750315
Value of chromozom [4, 5, 4, 0, 5, 3, 0, 0, 1, 1, 1, 2, 5, 0, 3, 4, 2, 6, 2, 6] : 264.82654275
Value of chromozom [4, 3, 5, 3, 3, 5, 0, 4, 1, 0, 3, 2, 4, 1, 2, 6, 1, 0, 2, 2] : 265.47669983
Value of chromozom [2, 0, 3, 1, 1, 0, 3, 2, 2, 5, 5, 1, 3, 0, 6, 4, 4, 4, 1, 5] : 266.099012691
Value of chromo

Generation Number :  0
Value of chromozom [2, 3, 0, 3, 1, 2, 0, 1, 3, 0, 3, 1, 4, 1, 0, 2, 1, 2, 1, 0] : 251.537205035
Value of chromozom [1, 0, 3, 0, 2, 1, 2, 1, 4, 0, 2, 1, 2, 0, 3, 2, 3, 1, 0, 3] : 253.998408553
Value of chromozom [2, 2, 3, 3, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2, 4, 3, 3, 2] : 254.993327281
Value of chromozom [3, 3, 0, 4, 2, 1, 2, 3, 1, 0, 1, 1, 3, 0, 2, 2, 0, 1, 2, 0] : 257.804445862
Value of chromozom [2, 3, 0, 0, 3, 3, 1, 2, 1, 4, 3, 1, 1, 0, 1, 0, 0, 2, 2, 2] : 259.030081549
Value of chromozom [3, 3, 1, 2, 2, 0, 3, 0, 4, 1, 2, 2, 1, 1, 2, 3, 0, 0, 2, 1] : 259.099102083
Value of chromozom [3, 2, 2, 2, 1, 0, 3, 2, 1, 1, 2, 0, 3, 1, 0, 3, 0, 2, 4, 1] : 260.861413158
Value of chromozom [2, 3, 1, 2, 0, 0, 3, 2, 1, 1, 1, 0, 3, 0, 3, 2, 0, 1, 2, 3] : 262.590828026
Value of chromozom [2, 2, 3, 3, 1, 1, 0, 3, 0, 3, 0, 1, 0, 1, 2, 0, 1, 2, 3, 2] : 263.161166517
Value of chromozom [1, 2, 2, 0, 3, 3, 0, 3, 2, 0, 1, 1, 1, 0, 3, 4, 1, 2, 0, 2] : 268.099896114
Value of chromozo

C:\Users\noamt\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\noamt\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in less
C:\Users\noamt\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in greater


Value of chromozom [1, 2, 2, 0, 5, 3, 0, 3, 2, 4, 0, 1, 0, 1, 2, 0, 1, 3, 4, 3] : 254.394951718
Value of chromozom [2, 2, 3, 3, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2, 4, 3, 3, 2] : 254.993327281
Value of chromozom [3, 3, 0, 4, 2, 1, 2, 3, 1, 0, 1, 1, 3, 0, 4, 2, 0, 1, 2, 3] : 255.218986884
Value of chromozom [1, 2, 2, 0, 5, 3, 1, 4, 2, 4, 0, 1, 0, 1, 2, 0, 1, 3, 3, 2] : 256.632037454
Value of chromozom [1, 3, 2, 0, 5, 3, 0, 3, 2, 4, 0, 2, 4, 2, 0, 2, 1, 3, 1, 1] : 256.912583749
Value of chromozom [1, 2, 2, 0, 4, 3, 0, 3, 2, 4, 0, 1, 0, 1, 2, 0, 1, 3, 3, 2] : 257.767853826
Value of chromozom [3, 3, 0, 4, 2, 1, 2, 3, 1, 0, 1, 1, 3, 0, 2, 2, 0, 1, 2, 0] : 257.804445862
Value of chromozom [3, 3, 0, 4, 2, 1, 2, 4, 2, 0, 1, 1, 0, 1, 2, 0, 1, 3, 3, 2] : 257.976146906
Value of chromozom [1, 2, 2, 0, 4, 3, 0, 3, 2, 5, 0, 1, 1, 1, 2, 0, 1, 3, 3, 2] : 256.816293637
the best solution is  [2, 3, 0, 3, 1, 2, 0, 1, 3, 0, 3, 1, 4, 1, 0, 2, 1, 2, 1, 0] with time 251.537205035 

Generation Number :  17
Val

KeyboardInterrupt: 

## Calculate N and Running

In [30]:
tvalue = stats.t.ppf(1-0.025,5)
final_list = []
plt.figure()


def calculate_N(tvalue, S):
    N= ((tvalue)**2)*(S**2)/(0.1**2)
    return N

def calculate_S(chromozom, parameters):
    result = []
    for i in range(5):
        new_result = f(chromozom, parameters)
        result.append(new_result)
    STD = np.std(result)
    return STD**2

best_solutions_list = [] #for each problem the best (chromozom, parameters)- from the main code
for i in range(len(best_solutions_list)):
    problem_results = []
    S = calculate_S(best_solutions_list[i][0], best_solutions_list[i][1])
    N = calculate_N(tvalue, S) 
    for j in range(N):
        problem_results.append(f(best_solutions_list[i][0], best_solutions_list[i][1]))
    problem_results.boxplot()
    final_list.append(np.average(problem_results))
plt.show()
print(final_list)

[]
